In [41]:
#读取数据
import torch
import os
import torchvision
def load_data():
    dataset=torchvision.datasets.FashionMNIST(root="fashion-mnist",train=False,transform=torchvision.transforms.ToTensor(),download=True)
    return dataset
#初始化数据集
dataset=load_data()
len(dataset),dataset,type(dataset)

(10000,
 Dataset FashionMNIST
     Number of datapoints: 10000
     Root location: fashion-mnist
     Split: Test
     StandardTransform
 Transform: ToTensor(),
 torchvision.datasets.mnist.FashionMNIST)

In [42]:
#数据集加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=1,
                                     shuffle=True,
                                     drop_last=True)

x, y = next(iter(loader))
len(loader), x[0,0,3], y

(10000,
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2353, 0.9333,
         0.8824, 0.8588, 0.9294, 0.9843, 0.5961, 0.3765, 0.9333, 0.9686, 0.8863,
         0.8549, 0.8980, 0.8431, 0.0078, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]),
 tensor([6]))

In [43]:
#nn.Linear实现RNN神经网络
class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.u = torch.nn.Linear(28, 256)
        self.w = torch.nn.Linear(256, 256)
        self.v = torch.nn.Linear(256, 10)
        self.tanh = torch.nn.Tanh()#激活函数
        self.LogSoftmax = torch.nn.LogSoftmax(dim=1)#定义归一化函数，对应了torch.nn.NLLLoss()

    def forward(self, inputs, hidden):
        
        u_x = self.u(inputs)#输入层
        hidden = self.w(hidden)
        hidden = self.tanh(hidden + u_x)#隐藏层和输入相加传入激活函数，并更新隐藏层
        output = self.LogSoftmax(self.v(hidden))#归一化输出
        
        return output, hidden

    def initHidden(self):#初始化隐藏层
        return torch.zeros(1, 256)

rnn=RNN()
rnn

RNN(
  (u): Linear(in_features=28, out_features=256, bias=True)
  (w): Linear(in_features=256, out_features=256, bias=True)
  (v): Linear(in_features=256, out_features=10, bias=True)
  (tanh): Tanh()
  (LogSoftmax): LogSoftmax(dim=1)
)

In [44]:
TP=FP=TN=FN=overall=0

def count(x,y,target_label):
    global TP,FP,TN,FN,overall
    if x==y:
        overall+=1
    if x==target_label and y==target_label:
        TP+=1
    elif x==target_label and y!=target_label:
        FP+=1    
    elif x!=target_label and y!=target_label:
        TN+=1        
    else:
        FN+=1        

def printParameter():
    global TP,FP,TN,FN,overall
    overallAccuracy=overall/(TP+TN+FP+FN)if (TP+TN+FP+FN) > 0 else 0
    accuracy=(TP+TN)/(TP+TN+FP+FN)if (TP+TN+FP+FN) > 0 else 0
    precision=TP/(TP+FP)if (TP + FP) > 0 else 0
    recall=TP/(TP+FN)if (TP+FN) > 0 else 0
    F1=2*(precision*recall)/(precision+recall)if (precision+recall) > 0 else 0
    print(f"overall accuracy={overallAccuracy}")
    print(f"accuracy={accuracy}")
    print(f"precision={precision}")
    print(f"recall={recall}")
    print(f"F1={F1}")

def resetCount():
    global TP,FP,TN,FN,overall
    TP=TN=FP=FN=overall=0
    

In [45]:
#训练
TP=TN=FP=FN=overall=0
target_label=0#自定义正例，其他为负例
rnn.cuda()
import tqdm
def train():
    optimizer = torch.optim.Adam(rnn.parameters(), lr=1e-4)
    criterion = torch.nn.NLLLoss()
    rnn.train()
    
    bar=tqdm.tqdm(range(5))
    hidden=rnn.initHidden().cuda()
    hidden = hidden.cuda()
    
    for epoch in bar:
        #rnn.zero_grad()
        for i ,( x , y )in enumerate(loader):
            
            x=x.cuda()
            y=y.cuda()
            for j in range(x.shape[2]):
                output, hidden = rnn(x[0,0,j], hidden)
            count(output.argmax(dim=1),y.item(),target_label)
            
            if i%100==0:
                print(i)
                printParameter()
                resetCount()
            
            loss = criterion(output,y)
            loss.backward(retain_graph=True)
            """在PyTorch中，当你执行.backward()方法时，计算图上的所有中间张量都会被释放，
            以节省内存。如果你需要多次进行反向传播，或者在反向传播后需要访问这些中间张量，
            你需要设置retain_graph=True参数。"""
            for p in rnn.parameters():
                p.data-=learning_rate*p.grad.data
            optimizer.zero_grad()

    return output,loss.item()

In [46]:
rnn = torch.load('model2')
learning_rate = 0.001
train()

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

0
overall accuracy=0.0
accuracy=0.0
precision=0.0
recall=0
F1=0
100
overall accuracy=0.34
accuracy=0.89
precision=0.45454545454545453
recall=0.5
F1=0.47619047619047616
200
overall accuracy=0.3
accuracy=0.8
precision=0.3157894736842105
recall=0.46153846153846156
F1=0.37499999999999994
300
overall accuracy=0.38
accuracy=0.88
precision=0.3333333333333333
recall=0.5
F1=0.4
400
overall accuracy=0.39
accuracy=0.91
precision=0.45454545454545453
recall=0.625
F1=0.5263157894736842
500
overall accuracy=0.39
accuracy=0.91
precision=0.75
recall=0.2727272727272727
F1=0.39999999999999997


  0%|                                                                                            | 0/5 [12:50<?, ?it/s]


KeyboardInterrupt: 

In [47]:
torch.save(rnn, 'model2')#硬件算力不够，就不训练了

In [48]:
#测试
import tqdm
@torch.no_grad()
def test():
    model = torch.load('model2')
    model.eval()
    model=model.cuda()
    hidden=rnn.initHidden()

    target_label=7#自定义正例，其他为负例
 
    #遍历整个数据集
    for i, (x, y) in enumerate(loader):
        x=x.cuda()
        y=y.cuda()
        hidden=hidden.cuda()
        for j in range(x.shape[2]):
            output, hidden = rnn(x[0,0,j], hidden)
                
        if i%1000==0:
            print(i)
        #求一个batch内各指标并累加
        target_label=7
        count(output.argmax(dim=1),y,target_label)
    printParameter()
    
test()


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
overall accuracy=0.31481666500149863
accuracy=0.7536217404336097
precision=0.28827267475447715
recall=0.998
F1=0.447333034513671
